# Devoir2 : SDA
** **


In [26]:
#Importation des bibliotheques
import pandas as pd
import hashlib
import mmh3
from bitarray import bitarray


On a implémenté 3 fonctions de hash, la première est un simple checksum (hash_checksum), la deuxième est une fonction de hash cryptographique (md5) qui a l'avantage d'être plus sécurisée (surtout dans ce cas ou on doit simuler un firewall), mais de l'autre cote, elle est moins rapide par rapport aux deux autres fonctions, la troisième est une fonction de hash non-cryptographique (mmh3) ce qui veut dire qu'elle est plus rapide mais faible en matière de sécurité.

In [27]:
#Checksum 
def hash_checksum(addr):
    addr = addr.replace(".","+")
    return eval(addr)

In [28]:
#Fonction 1 :crypto hash (slower/secure++)
def hash_md5(addr):
    
    h = hashlib.md5()
    h.update(bytes(addr, "utf-8"))
    a = int.from_bytes(h.digest(),"big")
    
    return a

In [29]:
#Fonction 2L:non-crypto hash (faster/secure--)
def hash_mmh3(addr):
    return mmh3.hash(addr,40)


In [30]:
#
def hash_4(addr):
    return


En ce qui concerne la taille des bitmaps, après consultation de la référence [1], si on choisit un taux (n/m) de bits par rapport aux nombre d'éléments (14.38) on doit avoir un taux de faux positifs de 0.1% si on suppose on utilise un k (nombre de fonctions de hachage) optimal, notre taux est 1000/70 = 14.28 mais le nombre de fonctions implémentées k=3 est inferieur a k optimal = m*log(2)/n. Du coup, le taux de faux positifs sera un peu plus élevé. Mais vu qu'on utilise des bitmaps séparés pour chaque fonction de hachage ça diminue beaucoup la possibilité d'avoir des collisions, et par conséquent moins de faux-positifs.

In [31]:
class BloomFilter : 
    def __init__(self,bitmapSize, hashFs,filterData):
        self.bitmapSize=bitmapSize
        self.hashFs=hashFs
        self.filterData=filterData
        b= bitarray(bitmapSize)
        b.setall(0)
        self.bitmaps = {}
        
        for i in range(len(hashFs)):
            self.bitmaps["bitmap{}".format(i)] = b.copy()
       
        self.fillBitmaps(self.filterData)

    def fillBitmaps(self,data):
              for item in data:
                      self.add(item)

    def add(self,addr):
        for i in range(len(self.hashFs)):
            index = self.hashFs[i](addr) % self.bitmapSize
            self.bitmaps["bitmap{}".format(i)][index] = 1

        
    
    def verify(self,addr):
        for i in range(len(self.hashFs)):
            index = self.hashFs[i](addr) % self.bitmapSize
            if self.bitmaps["bitmap{}".format(i)][index] == 0 : 
                return "l'adresse N'EST pas bloquee !"
            
        return "l'adresse est peut-etre bloquee"

    #Q3
    def VerifyMaybeAdress(self,addr):
        for item in self.filterData:
            if item == addr:
                return "L'adresse est 100'% 'bloqee !"
        
        return "l'adresse N'EST pas bloquee !"
     
    
    #Q4: Bonus
    def delete(self,addr):
        for i in range(len(self.filterData)):
            if self.filterData[i] == addr:
                self.filterData.drop(labels = i, inplace =True)
                self.filterData.reset_index(drop=True, inplace=True)
                self.reininitialize()
                #Supposition qu'il n'y a pas de duplications 
                return "Adresse supprimee"
        return "Cette adresse n'est pas bloquee"
        


    def reininitialize(self):
        self.__init__(self.bitmapSize,self.hashFs,self.filterData)


In [32]:
data1 = pd.read_table("adresses/addr1.dat",sep=" ", names=["Cmd","Addr"])
data2 = pd.read_table("adresses/addr2.dat",sep=" ", names=["Cmd","Addr"])

data = pd.concat([data1,data2])
data.reset_index(drop=True, inplace=True)
data.head()

,Cmd,Addr
0,T,63.213.250.249
1,T,73.88.188.199
2,T,230.78.246.129
3,T,63.70.29.207
4,T,208.218.65.201


In [33]:
data_B = data.where(cond = data["Cmd"] == "B")
data_B.dropna(inplace=True)
data_B.drop(columns=["Cmd"],inplace=True)
data_B.reset_index(drop=True, inplace=True)
data_B.head()

,Addr
0,220.47.220.177
1,118.251.47.146
2,148.32.96.48
3,18.79.84.124
4,233.181.186.37


In [34]:
data_D = data.where(cond = data["Cmd"] == "D")
data_D.dropna(inplace=True)
data_D.drop(columns=["Cmd"],inplace=True)
data_D.reset_index(drop=True, inplace=True)
data_D.head()

,Addr
0,203.202.91.97
1,132.151.214.38
2,84.197.207.33
3,110.89.92.174
4,220.47.220.177


In [35]:
data_T = data.where(cond = data["Cmd"] == "T")
data_T.dropna(inplace=True)
data_T.drop(columns=["Cmd"],inplace=True)
data_T.reset_index(drop=True, inplace=True)
data_T.head()

,Addr
0,63.213.250.249
1,73.88.188.199
2,230.78.246.129
3,63.70.29.207
4,208.218.65.201


In [36]:
bf = BloomFilter(1000,[hash_checksum,hash_mmh3,hash_md5],data_B["Addr"])

In [37]:
bf.filterData.head()


0    220.47.220.177
1    118.251.47.146
2      148.32.96.48
3      18.79.84.124
4    233.181.186.37
Name: Addr, dtype: object

In [38]:
bf.verify("220.47.220.177")

"l'adresse est peut-etre bloquee"

In [39]:
bf.VerifyMaybeAdress("220.47.220.177")

"L'adresse est 100'% 'bloqee !"

In [40]:
bf.verify("63.213.250.249")

"l'adresse N'EST pas bloquee !"

In [41]:
bf.bitmaps["bitmap0"].count()

67

In [42]:
bf.delete("220.47.220.177")

'Adresse supprimee'

In [43]:
bf.bitmaps["bitmap0"].count()

66

References:

1. https://corte.si/posts/code/bloom-filter-rules-of-thumb/
2. https://docs.python.org/3/library/hashlib.html#hashlib.algorithms_available
3. https://www.hackerearth.com/practice/data-structures/hash-tables/basics-of-hash-tables/tutorial/
4. https://www.maxburstein.com/blog/creating-a-simple-bloom-filter/
5. https://dridk.me/bloom-filter.html
6. https://bart.degoe.de/bloom-filters-bit-arrays-recommendations-caches-bitcoin/
7. https://crypto.stackexchange.com/questions/52734/need-for-secure-hash-function-for-a-bloom-filter